<a href="https://colab.research.google.com/github/lorenzopaoria/Smoking-detection-and-distance-analysis/blob/main/model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.utils as vutils
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from torchvision.io import read_image
from torchvision.ops import nms
from torchvision.utils import draw_bounding_boxes
import os
from PIL import Image
from pathlib import Path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
CLASS_NAMES = ["sigarette", "smoker", "non_smoker"]
MODEL_PATH = '/content/drive/MyDrive/pth_epoch/model_epoch_10'
OUTPUT_DIR = '/content/drive/MyDrive/test_trained'

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
class TestDataset(Dataset):
    def __init__(self, test_dir, transform=None):
        self.test_dir = Path(test_dir)
        self.image_files = list(self.test_dir.glob('*.[jp][pn][g]'))
        self.transform = transform or transforms.Compose([
            transforms.Resize((640, 640)),
            transforms.ToTensor(),
        ])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = str(self.image_files[idx])
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, img_path


In [ ]:
def load_model(model_path):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = torch.load(model_path, map_location=device)
    model.eval()
    return model, device

In [ ]:
def draw_predictions(image, boxes, labels, scores, output_path):
    """Disegna le bounding box sulle immagini e salva il risultato"""
    image = (image * 255).byte()

    colors = ["red", "green", "blue"]

    labels_text = [f"{CLASS_NAMES[int(l)]}: {s:.2f}" for l, s in zip(labels, scores)]

    image_with_boxes = draw_bounding_boxes(
        image.squeeze(0),
        boxes,
        labels=labels_text,
        colors=colors,
        width=2
    )

    image_with_boxes = image_with_boxes.permute(1, 2, 0).numpy()
    plt.figure(figsize=(12, 8))
    plt.imshow(image_with_boxes)
    plt.axis('off')
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
    plt.close()

In [ ]:
def evaluate_model(model, device, test_loader):
    """Valuta il modello e salva le immagini con le predizioni"""
    model.to(device)

    with torch.no_grad():
        for i, (images, img_paths) in enumerate(test_loader):
            images = images.to(device)

            predictions = model(images)

            for j, (prediction, img_path) in enumerate(zip(predictions, img_paths)):
                boxes = prediction['boxes']
                scores = prediction['scores']
                labels = prediction['labels']

                mask = scores > 0.5
                boxes = boxes[mask]
                scores = scores[mask]
                labels = labels[mask]

                if len(boxes) > 0:
                    output_filename = f"prediction_{i}_{j}.png"
                    output_path = os.path.join(OUTPUT_DIR, output_filename)

                    draw_predictions(
                        images[j].cpu(),
                        boxes.cpu(),
                        labels.cpu(),
                        scores.cpu(),
                        output_path
                    )

                    print(f"Saved predictions for {img_path} to {output_path}")

In [ ]:
if __name__ == "__main__":
    test_dataset = TestDataset('/content/drive/MyDrive/Photo/test')
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    model, device = load_model(MODEL_PATH)

    evaluate_model(model, device, test_loader)